In [17]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_vs_flat_debug'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


In [18]:
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["checksum"] = df["result.checksum"]

duckdb.sql(
    "SELECT ratio, dataset, COUNT (DISTINCT checksum) "
    "FROM df group by ratio, dataset "
    "ORDER BY ratio"
)

┌───────┬─────────┬──────────────────────────┐
│ ratio │ dataset │ count(DISTINCT checksum) │
│ int64 │ varchar │          int64           │
├───────┼─────────┼──────────────────────────┤
│     1 │ fb      │                        1 │
│    10 │ fb      │                        1 │
│   100 │ fb      │                        1 │
│  1000 │ fb      │                        1 │
└───────┴─────────┴──────────────────────────┘